In [68]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [69]:
training_data = pd.read_csv('train.csv')


In [70]:
#CryoSleep and VIP = boolean
#destination and home planet = groups
#cabin needs to be split into 3 separate columns
print(training_data['Destination'].unique())
'''training_data['CryoSleep'] = training_data['CryoSleep'].map({'True': 1, 'False': 0})
training_data['VIP'] = training_data['VIP'].map({'True': 1, 'False': 0})
'''
training_data['HomePlanet'] = training_data['HomePlanet'].map({'Europa': 0, 'Earth': 1, 'Mars':2})
training_data['Destination'] = training_data['Destination'].map({'TRAPPIST-1e': 0, 'PSO J318.5-22': 1, '55 Cancri e':2})
training_data[['Deck', 'Num', 'Side']] = training_data['Cabin'].str.split('/', expand=True)
training_data = training_data.drop(columns=['Cabin'])
deck_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'T': 7}
training_data['Deck'] = training_data['Deck'].map(deck_map)
training_data['Side'] = training_data['Side'].map({'P': 0, 'S': 1})
training_data['CryoSleep'] = training_data['CryoSleep'].fillna(False).astype(int)
training_data['VIP'] = training_data['VIP'].fillna(False).astype(int)
training_data['Transported'] = training_data['Transported'].fillna(False).astype(int)


training_data


['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]


,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side
0,0001_01,0.0,0,0.0,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,1.0,0,0.0
1,0002_01,1.0,0,0.0,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,5.0,0,1.0
2,0003_01,0.0,0,0.0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,0.0,0,1.0
3,0003_02,0.0,0,0.0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,0.0,0,1.0
4,0004_01,1.0,0,0.0,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,5.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,0,2.0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,0.0,98,0.0
8689,9278_01,1.0,1,1.0,18.0,0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,6.0,1499,1.0
8690,9279_01,1.0,0,0.0,26.0,0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,6.0,1500,1.0
8691,9280_01,0.0,0,2.0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,4.0,608,1.0


In [71]:
from sklearn.preprocessing import OrdinalEncoder
deck_encoder = OrdinalEncoder()
side_encoder = OrdinalEncoder()

In [72]:
training_data['Deck'] = deck_encoder.fit_transform(training_data[['Deck']])
training_data['Side'] = side_encoder.fit_transform(training_data[['Side']])
training_data

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side
0,0001_01,0.0,0,0.0,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,1.0,0,0.0
1,0002_01,1.0,0,0.0,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,5.0,0,1.0
2,0003_01,0.0,0,0.0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,0.0,0,1.0
3,0003_02,0.0,0,0.0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,0.0,0,1.0
4,0004_01,1.0,0,0.0,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,5.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,0,2.0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,0.0,98,0.0
8689,9278_01,1.0,1,1.0,18.0,0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,6.0,1499,1.0
8690,9279_01,1.0,0,0.0,26.0,0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,6.0,1500,1.0
8691,9280_01,0.0,0,2.0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,4.0,608,1.0


In [73]:

target = training_data['Transported']
training_data = training_data.drop(columns=['Transported', 'PassengerId','Name'])

training_data

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,0.0,0,0.0,39.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0
1,1.0,0,0.0,24.0,0,109.0,9.0,25.0,549.0,44.0,5.0,0,1.0
2,0.0,0,0.0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0.0,0,1.0
3,0.0,0,0.0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0.0,0,1.0
4,1.0,0,0.0,16.0,0,303.0,70.0,151.0,565.0,2.0,5.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,0,2.0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0.0,98,0.0
8689,1.0,1,1.0,18.0,0,0.0,0.0,0.0,0.0,0.0,6.0,1499,1.0
8690,1.0,0,0.0,26.0,0,0.0,0.0,1872.0,1.0,0.0,6.0,1500,1.0
8691,0.0,0,2.0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,4.0,608,1.0


In [74]:

features  = training_data.columns.tolist()
imputer = IterativeImputer(random_state=42)
X_train_imputed = imputer.fit_transform(training_data)

X_imputed_df = pd.DataFrame(X_train_imputed, columns=features)

preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), features)

        ]
    )

processed_data = preprocessor.fit_transform(X_imputed_df)

X_train, X_test, y_train, y_test = train_test_split(processed_data, target, test_size=0.1, random_state=42, shuffle=False)
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)


GradientBoostingRegressor(random_state=42)

In [124]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
y_pred = model.predict(X_test)
# Convert predictions to 0/1 if needed
def test(y_pred, y_test):
    y_pred = np.array(y_pred)

    # ✅ If shape is (n_samples, 1), flatten it to (n_samples,)
    if y_pred.ndim > 1:
        y_pred = y_pred.ravel()

    # ✅ Only threshold if values are probabilities (i.e. float)
    if y_pred.dtype != int and y_pred.dtype != np.int64:
        y_pred = (y_pred >= 0.5).astype(int)

    np_y_test = np.array(y_test)
    print(np.unique(np_y_test), np_y_test.dtype)
    print(np.unique(y_pred), y_pred.dtype)


    acc = accuracy_score(np_y_test, y_pred)
    prec = precision_score(np_y_test, y_pred)
    rec = recall_score(np_y_test, y_pred)
    f1 = f1_score(np_y_test, y_pred)

    print(f"\nAccuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:\n", confusion_matrix(np_y_test, y_pred))

test(y_pred, y_test)


[0 1] int64
[0 1] int64

Accuracy: 0.8149
Precision: 0.7816
Recall: 0.8374
F1 Score: 0.8086
Confusion Matrix:
 [[369  95]
 [ 66 340]]


In [121]:
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from catboost import CatBoostClassifier

def find_best_model_prediction(X_all, y_all):
    numeric_features = X_all.columns.tolist()
    X_train_imputed = imputer.fit_transform(X_all)

    X_imputed_df = pd.DataFrame(X_train_imputed, columns=numeric_features)
    preprocessor = ColumnTransformer(
        transformers=[('num', StandardScaler(), numeric_features)]
    )

    processed_data = preprocessor.fit_transform(X_imputed_df)

    models = {
        "GradientBoostingClassifier": (
            GradientBoostingClassifier(random_state=42),
            {
                'n_estimators': [100,200],
                'max_depth': [3, 6],
                'learning_rate': [0.05, 0.01],
                'min_samples_split': [2,5],
                'min_samples_leaf': [1,3],
                'subsample': [0.6],
                'max_features': ['sqrt']
            }
        ),
        "HistGradientBoosting": (
            HistGradientBoostingClassifier(random_state=42, early_stopping=True, validation_fraction=0.1, n_iter_no_change=10),
            {
                'max_iter': [100,200],
                'max_depth': [3, 6],
                'learning_rate': [0.05, 0.01],
                'min_samples_leaf': [10, 20, 30],
                'l2_regularization': [0.1, 0.5]
            }
        ),
        "RandomForest": (
            RandomForestClassifier(random_state=42),
            {
                'n_estimators': [100,200],
                'max_depth': [10, 15],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [5, 3],
                'max_features': ['sqrt']
            }
        ),
        "MLPClassifier": (
            MLPClassifier(random_state=42, max_iter=5000,early_stopping=True,validation_fraction=0.1, n_iter_no_change=10),
            {
                'hidden_layer_sizes': [(50, 50)],
                'activation': ['relu'],
                'learning_rate_init': [0.001, 0.01],
                'learning_rate': [ 'adaptive'],
                'solver': ['adam'],
                'alpha': [ 0.01,0.1]
            }
        ),
        "XGBoost": (
            XGBClassifier(random_state=42, verbosity=0 ),
            {
                'n_estimators': [100,200],
                'max_depth': [3, 6],
                'learning_rate': [0.05, 0.01],
                'subsample': [0.6, 0.8],
                'reg_alpha': [0.1, 0.5],
                'min_child_weight': [3,5],
                'gamma': [0.1, 0.3],
                'colsample_bytree': [0.8, 1]
            }
        ),
        "Catboost":(
            CatBoostClassifier(verbose=0, random_state=42),
            {
                'iterations': [100,200],
                'depth': [3, 6],
                'learning_rate': [0.05, 0.01],
                'l2_leaf_reg': [5, 7],
                'random_strength': [1,3],
            }
        )
    }

    X_train, X_test, y_train, y_test = train_test_split(processed_data, y_all, test_size=0.15, random_state=42, shuffle=True)
    
    results =[]

    for name, (model, param_grid) in models.items():
        print(f"\nTuning {name}...")
        grid = GridSearchCV(model, param_grid, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='accuracy', n_jobs=-1)
        grid.fit(X_train, y_train)

        print("Best parameters:", grid.best_params_)
        print("Best score on training data:", grid.best_score_)

        y_pred = grid.best_estimator_.predict(X_test)

        y_pred = np.round(y_pred).astype(int)
        np_y_test = np.array(y_test).astype(int)

        y_pred = np.array(y_pred)
        np_y_test = np.array(y_test)
        print(np.unique(np_y_test), np_y_test.dtype)
        print(np.unique(y_pred), y_pred.dtype)


        acc = accuracy_score(np_y_test, y_pred)
        prec = precision_score(np_y_test, y_pred, pos_label=1)
        rec = recall_score(np_y_test, y_pred, pos_label=1)
        f1 = f1_score(np_y_test, y_pred, pos_label=1)

        print(f"\nAccuracy: {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall: {rec:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print("Confusion Matrix:\n", confusion_matrix(np_y_test, y_pred))

        results.append({
            'name': name,
            'estimator': grid.best_estimator_,
            'accuracy': acc,
            'y_test': y_test,
            'y_pred': y_pred
        })

    top3 = sorted(results, key=lambda x: x['accuracy'], reverse=True)[:3]

    print("\n🏆 Top 3 models:")
    for i, result in enumerate(top3, start=1):
        print(f"{i}. {result['name']} - Accuracy: {result['accuracy']:.4f}")


    print(f"\n✅ Best model: {top3[0]['name']} with Accuracy: {top3[0]['accuracy']:.4f}")
    return [(r['name'], r['estimator'], r['y_test'], r['y_pred']) for r in top3], preprocessor

In [122]:
top3_models, preprocessor = find_best_model_prediction(training_data, target)


Tuning GradientBoostingClassifier...
Best parameters: {'learning_rate': 0.05, 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.6}
Best score on training data: 0.8105264942011153
[0 1] int64
[0 1] int64

Accuracy: 0.7876
Precision: 0.7681
Recall: 0.8192
F1 Score: 0.7928
Confusion Matrix:
 [[497 160]
 [117 530]]

Tuning HistGradientBoosting...
Best parameters: {'l2_regularization': 0.5, 'learning_rate': 0.05, 'max_depth': 6, 'max_iter': 200, 'min_samples_leaf': 30}
Best score on training data: 0.8093088154590505
[0 1] int64
[0 1] int64

Accuracy: 0.7952
Precision: 0.7870
Recall: 0.8053
F1 Score: 0.7960
Confusion Matrix:
 [[516 141]
 [126 521]]

Tuning RandomForest...
Best parameters: {'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 200}
Best score on training data: 0.8094437669891883
[0 1] int64
[0 1] int64

Accuracy: 0.7937
Precision: 0.7779
Recall: 0.8176
F

In [113]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

def stack_models(top3_models, X_all, y_all):
    estimators = [(name, model) for name, model, _, _ in top3_models]

    # Use Logistic Regression as final estimator (you can change this)
    stacked_model = StackingClassifier(
        estimators=estimators,
        final_estimator=RandomForestClassifier(n_estimators=100, max_depth=5),
        passthrough=False,
        cv=5,
        n_jobs=-1
    )

    # Reuse X_all and y_all to fit stacked model
    # You must reapply preprocessing to the full dataset
    numeric_features = X_all.columns.tolist()
    X_train_imputed = imputer.fit_transform(X_all)

    X_imputed_df = pd.DataFrame(X_train_imputed, columns=numeric_features)
    preprocessor = ColumnTransformer(
        transformers=[('num', StandardScaler(), numeric_features)]
    )

    X_all_scaled_array = preprocessor.fit_transform(X_imputed_df)
    X_all_scaled = pd.DataFrame(X_all_scaled_array, columns=numeric_features)

    X_train, X_test, y_train, y_test = train_test_split(X_all_scaled, y_all, test_size=0.15, random_state=42, shuffle=True)
    
    # Fit
    stacked_model.fit(X_train, y_train)
    y_pred = stacked_model.predict(X_test)

    test(y_pred, y_test)
    return stacked_model


In [123]:
stacked_model = stack_models(top3_models, training_data, target)

[0 1] int64
[0 1] int64

Accuracy: 0.7983
Precision: 0.7824
Recall: 0.8223
F1 Score: 0.8018
Confusion Matrix:
 [[509 148]
 [115 532]]


In [85]:
import joblib

# Save the model
joblib.dump(best_overall, "best_model.pkl")

# Optionally also save the preprocessor if you need it later
joblib.dump(preprocessor, "preprocessor.pkl")


['preprocessor.pkl']

In [86]:
model = joblib.load("best_model.pkl")
preprocessor = joblib.load("preprocessor.pkl")

In [98]:
def clean_data(data):
    training_data = data.copy()
    
    training_data['HomePlanet'] = training_data['HomePlanet'].map({'Europa': 0, 'Earth': 1, 'Mars':2})
    training_data['Destination'] = training_data['Destination'].map({'TRAPPIST-1e': 0, 'PSO J318.5-22': 1, '55 Cancri e':2})
    training_data[['Deck', 'Num', 'Side']] = training_data['Cabin'].str.split('/', expand=True)
    training_data = training_data.drop(columns=['Cabin'])
    deck_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'T': 7}
    training_data['Deck'] = training_data['Deck'].map(deck_map)
    training_data['Side'] = training_data['Side'].map({'P': 0, 'S': 1})
    training_data['CryoSleep'] = training_data['CryoSleep'].fillna(False).astype(int)
    training_data['VIP'] = training_data['VIP'].fillna(False).astype(int)

    training_data = training_data.drop(columns=['Name'])
    X_imputed = imputer.transform(training_data)
    training_data = pd.DataFrame(X_imputed, columns=training_data.columns)

    '''training_data['Deck'] = deck_encoder.transform(training_data[['Deck']])
    training_data['Side'] = side_encoder.transform(training_data[['Side']])'''
    return training_data


In [93]:
def create_submission(data, model, preprocessor):
    passenger_ids = data['PassengerId']
    features = data.drop(columns=['PassengerId'])
    run_data = clean_data(features)
    X_test_processed = preprocessor.transform(run_data)
    predictions = model.predict(X_test_processed)
    submission = pd.DataFrame({
        'PassengerId': passenger_ids,
        'Transported': predictions.astype(bool)
    })
    submission.to_csv('submission.csv', index=False)

In [99]:
data = pd.read_csv('test.csv')
create_submission(data,model,preprocessor)